Буду работать с данными https://www.kaggle.com/datasets/lokeshparab/amazon-products-dataset/
моя основная задача понять как работает ранжирование и написать простенькую програму локального поискавика по данным

In [4]:
import pandas as pd
import os

In [2]:
!ls -la

total 81632
drwxr-xr-x 1 root root    12288 Oct 18 19:55 .
drwxr-xr-x 1 root root     4096 Oct 18 19:38 ..
-rw-r--r-- 1 root root 83555296 Oct 18 19:49 archive.zip
drwxr-xr-x 4 root root     4096 Oct 16 13:41 .config
drwxr-xr-x 2 root root     4096 Oct 18 19:50 .ipynb_checkpoints
drwxr-xr-x 1 root root     4096 Oct 16 13:41 sample_data


In [3]:
!unzip "/content/archive.zip" -d "/content/data"

Archive:  /content/archive.zip
  inflating: /data/Air Conditioners.csv  
  inflating: /data/All Appliances.csv  
  inflating: /data/All Books.csv     
  inflating: /data/All Car and Motorbike Products.csv  
  inflating: /data/All Electronics.csv  
  inflating: /data/All English.csv   
  inflating: /data/All Exercise and Fitness.csv  
  inflating: /data/All Grocery and Gourmet Foods.csv  
  inflating: /data/All Hindi.csv     
  inflating: /data/All Home and Kitchen.csv  
  inflating: /data/All Movies and TV Shows.csv  
  inflating: /data/All Music.csv     
  inflating: /data/All Pet Supplies.csv  
  inflating: /data/All Sports Fitness and Outdoors.csv  
  inflating: /data/All Video Games.csv  
  inflating: /data/Amazon Fashion.csv  
  inflating: /data/Amazon Pharmacy.csv  
  inflating: /data/Amazon-Products.csv  
  inflating: /data/Baby Bath Skin and Grooming.csv  
  inflating: /data/Baby Fashion.csv  
  inflating: /data/Baby Products.csv  
  inflating: /data/Backpacks.csv     
  inflat

In [5]:
folder_path = '/content/data'
df = pd.DataFrame()

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        full_path = os.path.join(folder_path, file)
        data = pd.read_csv(full_path)
        df = pd.concat([df, data], ignore_index=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103170 entries, 0 to 1103169
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   name            1103170 non-null  object 
 1   main_category   1103170 non-null  object 
 2   sub_category    1103170 non-null  object 
 3   image           1103170 non-null  object 
 4   link            1103170 non-null  object 
 5   ratings         751582 non-null   object 
 6   no_of_ratings   751582 non-null   object 
 7   discount_price  980844 non-null   object 
 8   actual_price    1067544 non-null  object 
 9   Unnamed: 0      551585 non-null   float64
dtypes: float64(1), object(9)
memory usage: 84.2+ MB


In [7]:
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Protoner 16Kg PVC Combo Home Gym Package,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71K-gcsnOY...,https://www.amazon.in/Protoner-16Kg-PVC-Combo-...,3.6,917,₹965.08,"₹4,999",NaN
1,Unisex Hexagon Rubber Vinyl Fixed Dumbbell - P...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71pcrSXprs...,https://www.amazon.in/Generic-Vinyl-Coated-Iro...,4.4,367,₹389,"₹6,450",NaN
2,LAFILLETTE 150 lbs Metal Hand Gripper Forearm ...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41oTg0XNbF...,https://www.amazon.in/LALA-LIFE-Gripper-Streng...,4.5,381,₹489,"₹1,199",NaN
3,Aurion Genuine Leather Pro Weight Lifting Belt...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/813g2hRYuo...,https://www.amazon.in/Genuine-Comfortable-Adju...,4.4,446,₹799,₹899,NaN
4,SPIRO PVC (Round/Hex) Dumbbells (1 Kg. X 2 = 2...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/61VcW8otph...,https://www.amazon.in/Dumbbells-Equipment-Fitn...,3.5,52,₹82.31,₹200,NaN


In [13]:
from IPython.display import Image, display
from random import randint

for i in range(3):
  index = randint(0, len(df))
  url = df.iloc[index]['image']
  display(Image(url=url, width=300))

Сначала поработаю с текстовыми признаками с помощью bm25

---


$$
BM25(q, d) = \sum_{i=1}^{n} IDF(q_i) \times \frac{TF(q_i, d) \times (k_1 + 1)}{TF(q_i, d) + k_1 \times \left(1 - b + b \times \frac{|d|}{avgdl}\right)}
$$

$TF(q_i, d)$ - частота слова $q_i$ в документе

$avgdl$ - средняя длина документов

$$
IDF(q_i) = \log \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5}
$$

$n(q_i)$ - количество документов содержащих $q_i$.

$N$ - общее число документов.

---


bm25 это доработанная версия TF-IDF с помощью дополнительных коэфициентов мы отдаем больший приоритет коротким названиям и штрафуем за множественное повторение одного слова

In [15]:
!pip install rank_bm25

In [16]:
import numpy as np
import string

# Handle missing values
df['name'] = df['name'].fillna('')

# Tokenize, lowercase, and remove punctuation
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    return tokens

df['tokenized_name'] = df['name'].apply(preprocess_text)

display(df.head())

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0,tokenized_name
0,Protoner 16Kg PVC Combo Home Gym Package,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71K-gcsnOY...,https://www.amazon.in/Protoner-16Kg-PVC-Combo-...,3.6,917,₹965.08,"₹4,999",NaN,"[protoner, 16kg, pvc, combo, home, gym, package]"
1,Unisex Hexagon Rubber Vinyl Fixed Dumbbell - P...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/71pcrSXprs...,https://www.amazon.in/Generic-Vinyl-Coated-Iro...,4.4,367,₹389,"₹6,450",NaN,"[unisex, hexagon, rubber, vinyl, fixed, dumbbe..."
2,LAFILLETTE 150 lbs Metal Hand Gripper Forearm ...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/41oTg0XNbF...,https://www.amazon.in/LALA-LIFE-Gripper-Streng...,4.5,381,₹489,"₹1,199",NaN,"[lafillette, 150, lbs, metal, hand, gripper, f..."
3,Aurion Genuine Leather Pro Weight Lifting Belt...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/813g2hRYuo...,https://www.amazon.in/Genuine-Comfortable-Adju...,4.4,446,₹799,₹899,NaN,"[aurion, genuine, leather, pro, weight, liftin..."
4,SPIRO PVC (Round/Hex) Dumbbells (1 Kg. X 2 = 2...,sports & fitness,Strength Training,https://m.media-amazon.com/images/I/61VcW8otph...,https://www.amazon.in/Dumbbells-Equipment-Fitn...,3.5,52,₹82.31,₹200,NaN,"[spiro, pvc, roundhex, dumbbells, 1, kg, x, 2,..."


In [21]:
from rank_bm25 import BM25Okapi

tokenized_products = [t_p for t_p in df['tokenized_name']]
bm25 = BM25Okapi(tokenized_products)

In [24]:
queries = [
    "smartphone realme new 8/256",
    "glue",
    "batterfly"
]

In [25]:
for query in queries:
    tokenized_query = preprocess_text(query)
    scores = bm25.get_scores(tokenized_query)
    print(f"\nЗапрос: '{query}'")
    for (product, score, index) in sorted(zip(df['name'], scores, range(len(df))), key=lambda x: x[1], reverse=True)[:1]:
        print(f"  {product} -> BM25: {score:.3f}")
        url = df.iloc[index]['image']
        display(Image(url=url, width=300))


Запрос: 'smartphone realme new 8/256'
  Siwi Fast Charger for Realme 1, Realme 2, Realme 2 Pro, Realme 3, Realme 3 Pro, Realme 3i, Realme 4, Realme 5, Realme C1 2... -> BM25: 12.804



Запрос: 'glue'
  SOLDTRUE Huda Beauty Eye-Lash Glue False Eyelash Adhesive Glue Eyelashes Extension Glue 1pc -> BM25: 13.546



Запрос: 'batterfly'
  NiyatiFab Purple Color Dual Batterfly with Black Color Small Heart Combo Watch - for Girls -> BM25: 11.578
